#Model 4: Training yolov8n-seg.pt on yolov5(pytorch) format dataset


In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="PMrUjKgeSmZTnHqIYsdQ")
project = rf.workspace("myworspace-x66mw").project("pallet-detection-and-counting-zejdg")
version = project.version(2)
dataset = version.download("yolov5")


loading Roboflow workspace...
loading Roboflow project...


In [ ]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device name: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'No GPU'}")

CUDA available: True
CUDA device name: Tesla T4


In [ ]:
!pip install ultralytics
import os
from ultralytics import YOLO
from google.colab import drive
drive.mount('/content/drive')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 24.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 68.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

In [ ]:
# Using the 'dataset' variable directly as requested
dataset_base_dir = dataset.location # Get the local path to the downloaded dataset
data_yaml_path = os.path.join(dataset_base_dir, 'data.yaml')

print(f"Dataset downloaded to: {dataset_base_dir}")
print(f"data.yaml located at: {data_yaml_path}")

Dataset downloaded to: /content/Pallet-Detection-and-Counting-2
data.yaml located at: /content/Pallet-Detection-and-Counting-2/data.yaml


In [ ]:
print("\nLoading pre-trained YOLOv8n-seg model...")
model = YOLO('yolov8n-seg.pt') # Using the nano segmentation model
print("YOLOv8n-seg model loaded successfully.")


Loading pre-trained YOLOv8n-seg model...
YOLOv8n-seg model loaded successfully.


In [ ]:
print(f"DEBUG: Value of data_yaml_path: {data_yaml_path}")
if os.path.exists(data_yaml_path):
    print(f"DEBUG: data.yaml file EXISTS at this path.")

else:
    print(f"DEBUG: data.yaml file DOES NOT EXIST at this path. This is a problem.")

DEBUG: Value of data_yaml_path: /content/Pallet-Detection-and-Counting-2/data.yaml
DEBUG: data.yaml file EXISTS at this path.


In [ ]:
import os
import yaml

# Assuming dataset_base_dir and data_yaml_path are already defined

print(f"Attempting to read data.yaml from: {data_yaml_path}")

if os.path.exists(data_yaml_path):
    with open(data_yaml_path, 'r') as f:
        data_yaml_content = yaml.safe_load(f)

    print("\n--- Contents of data.yaml ---")
    print(yaml.dump(data_yaml_content, default_flow_style=False))
    print("-----------------------------\n")
else:
    print(f"Error: data.yaml not found at {data_yaml_path}. Please ensure the dataset is downloaded.")

Attempting to read data.yaml from: /content/Pallet-Detection-and-Counting-2/data.yaml

--- Contents of data.yaml ---
names:
- Pallet
nc: 1
roboflow:
  license: CC BY 4.0
  project: pallet-detection-and-counting-zejdg
  url: https://universe.roboflow.com/myworspace-x66mw/pallet-detection-and-counting-zejdg/dataset/2
  version: 2
  workspace: myworspace-x66mw
test: ./test/images
train: ./train/images
val: ./valid/images

-----------------------------



In [ ]:
# CRITICAL: Adjust paths in data.yaml for Ultralytics compatibility ---
# Roboflow sometimes generates data.yaml with paths that are too deep or absolute
# for Ultralytics when the dataset is downloaded into a subdirectory.
print(f"\nAdjusting paths in {data_yaml_path}...")

with open(data_yaml_path, 'r') as f:
    data_yaml_content = yaml.safe_load(f)

# The `path` field in data.yaml, if present, can also cause issues if it's absolute
# or points to a problematic location. Setting it to './' (current directory) is safest.
if 'path' in data_yaml_content:
    original_yaml_path_field = data_yaml_content['path']
    # If the path field is absolute or contains the dataset folder name, correct it.
    # The dataset_base_dir is already the root where data.yaml sits.
    if original_yaml_path_field.startswith('/') or os.path.basename(dataset_base_dir) in original_yaml_path_field:
        data_yaml_content['path'] = './' # Set to current directory relative to data.yaml
        print(f"  Corrected 'path' field from '{original_yaml_path_field}' to './'")
    else:
        print(f"  'path' field '{original_yaml_path_field}' seems correct, no change needed.")
else:
    print("  'path' field not found in data.yaml. Assuming relative paths from data.yaml's location.")


# Correct 'train', 'val', 'test' paths to be relative to the data.yaml's location
# They should typically point directly to 'train/images', 'valid/images', 'test/images'
# within the dataset_base_dir.
dataset_folder_name = os.path.basename(dataset_base_dir) # e.g., 'Pallet-Detection-and-Counting-2'

for key in ['train', 'val', 'test']:
    if key in data_yaml_content and isinstance(data_yaml_content[key], str):
        original_path = data_yaml_content[key]
        corrected_path = original_path

        # Remove leading '../' if present
        if corrected_path.startswith('../'):
            corrected_path = corrected_path[3:]

        # Remove the duplicated dataset folder name if it's at the beginning of the path
        if corrected_path.startswith(f"{dataset_folder_name}/"):
            corrected_path = corrected_path[len(f"{dataset_folder_name}/"):]

        # Ensure it's a relative path starting with './' if it's just a folder name
        if not corrected_path.startswith('./') and not corrected_path.startswith('/'):
            corrected_path = f'./{corrected_path}'

        if original_path != corrected_path:
            print(f"  Corrected '{key}' path from '{original_path}' to '{corrected_path}'")
            data_yaml_content[key] = corrected_path
        else:
            print(f"  '{key}' path '{original_path}' seems correct, no change needed.")

# Write the corrected YAML content back to the file
with open(data_yaml_path, 'w') as f:
    yaml.dump(data_yaml_content, f, default_flow_style=False)

print("Paths in data.yaml adjusted successfully.")
# --- End of CRITICAL adjustment ---



Adjusting paths in /content/Pallet-Detection-and-Counting-2/data.yaml...
  'path' field not found in data.yaml. Assuming relative paths from data.yaml's location.
  'train' path './train/images' seems correct, no change needed.
  'val' path './valid/images' seems correct, no change needed.
  'test' path './test/images' seems correct, no change needed.
Paths in data.yaml adjusted successfully.


In [ ]:
# Configure and Start Training ---
print("\nStarting model training...")

# Define training parameters
# 'data': Path to your data.yaml file
# 'epochs': Number of training epochs. Start with a moderate number (e.g., 50-100).
# 'imgsz': Input image size. Keep it consistent with your dataset's image sizes.
# 'batch': Batch size. Adjust based on your GPU memory (lower if out of memory).
# 'name': Name for your training run, results will be saved in runs/segment/your_name
# 'device': '0' for GPU (if available), 'cpu' for CPU.
# 'patience': Number of epochs to wait for no improvement before early stopping (optional)

results = model.train(
    data=data_yaml_path, # This is the corrected data.yaml path
    epochs=50,       # <<<--- ADJUST EPOCHS AS NEEDED
    imgsz=640,       # <<<--- ADJUST IMAGE SIZE IF YOUR DATASET IS DIFFERENT
    batch=16,        # <<<--- ADJUST BATCH SIZE BASED ON GPU MEMORY
    name='pallet_yolov8n_segmentation_run', # <<<--- CUSTOMIZE RUN NAME
    device=0,        # Use GPU (0 for first GPU), or 'cpu' if no GPU
    # patience=10    # Uncomment for early stopping
)

print("\nTraining complete! Results saved to the 'runs/segment' directory.")
print(f"Best model saved at: {model.trainer.best}")



Starting model training...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/Pallet-Detection-and-Counting-2/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-seg.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=pallet_yolov8n_segmentation_run2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overl

train: Scanning /content/Pallet-Detection-and-Counting-2/train/labels... 465 images, 0 backgrounds, 0 corrupt: 100%|██████████| 465/465 [00:01<00:00, 319.00it/s]


train: New cache created: /content/Pallet-Detection-and-Counting-2/train/labels.cache
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, method='weighted_average', num_output_channels=3), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 35.7±36.1 MB/s, size: 59.9 KB)


val: Scanning /content/Pallet-Detection-and-Counting-2/valid/labels... 31 images, 0 backgrounds, 0 corrupt: 100%|██████████| 31/31 [00:00<00:00, 166.65it/s]

val: /content/Pallet-Detection-and-Counting-2/valid/images/chep-wooden-pallet-BJ229W_jpg.rf.c05b3c41900a53e79796c3219e4735a1.jpg: 3 duplicate labels removed


...


50 epochs completed in 0.352 hours.
Optimizer stripped from runs/segment/pallet_yolov8n_segmentation_run2/weights/last.pt, 6.8MB
Optimizer stripped from runs/segment/pallet_yolov8n_segmentation_run2/weights/best.pt, 6.8MB

Validating runs/segment/pallet_yolov8n_segmentation_run2/weights/best.pt...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/1 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.57it/s]


                   all         31       1013      0.907      0.964      0.967      0.683      0.845        0.9      0.884      0.364
Speed: 0.2ms preprocess, 2.8ms inference, 0.0ms loss, 2.4ms postprocess per image
Results saved to runs/segment/pallet_yolov8n_segmentation_run2

Training complete! Results saved to the 'runs/segment' directory.
Best model saved at: runs/segment/pallet_yolov8n_segmentation_run2/weights/best.pt


In [ ]:
from google.colab import files
files.download('/content/runs/segment/pallet_yolov8n_segmentation_run2/weights/best.pt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# --- 7. Evaluate the Trained Model on the Test Set ---
print("\nEvaluating the trained model on the test set...")

# The .val() method performs the validation/evaluation.
# 'split="test"' ensures evaluation is performed on the test split defined in data.yaml.
evaluation_results = model.val(data=data_yaml_path, split="test")



Evaluating the trained model on the test set...
Ultralytics 8.3.163 🚀 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8n-seg summary (fused): 85 layers, 3,258,259 parameters, 0 gradients, 12.0 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 128.9±145.3 MB/s, size: 42.0 KB)


val: Scanning /content/Pallet-Detection-and-Counting-2/test/labels... 23 images, 0 backgrounds, 0 corrupt: 100%|██████████| 23/23 [00:00<00:00, 379.07it/s]

val: New cache created: /content/Pallet-Detection-and-Counting-2/test/labels.cache



                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):   0%|          | 0/2 [00:00<?, ?it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95):  50%|█████     | 1/2 [00:00<00:00,  1.00it/s]

WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...
WARNING ⚠️ Limiting validation plots to first 50 items per image for speed...


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:01<00:00,  1.15it/s]


                   all         23        589      0.932      0.951      0.969      0.661      0.923      0.881      0.916      0.383
Speed: 0.3ms preprocess, 25.5ms inference, 0.0ms loss, 2.9ms postprocess per image
Results saved to runs/segment/pallet_yolov8n_segmentation_run22


In [ ]:
print("\n--- Detailed Evaluation Results ---")

# Accessing Bounding Box (Box) Metrics
print("\nBounding Box Metrics:")
print(f"  Precision (P): {evaluation_results.box.p.item():.3f}")
print(f"  Recall (R): {evaluation_results.box.r.item():.3f}")
print(f"  F1 Score: {2 * (evaluation_results.box.p.item() * evaluation_results.box.r.item()) / (evaluation_results.box.p.item() + evaluation_results.box.r.item()):.3f}")
print(f"  mAP50 (Mean Average Precision at IoU=0.50): {evaluation_results.box.map50.item():.3f}")
print(f"  mAP50-95 (Mean Average Precision across IoU=0.50 to 0.95): {evaluation_results.box.map.item():.3f}")

# Accessing Segmentation Mask (Mask) Metrics
print("\nSegmentation Mask Metrics:")
if hasattr(evaluation_results, 'mask') and evaluation_results.mask is not None:
    print(f"  Precision (P): {evaluation_results.mask.p.item():.3f}")
    print(f"  Recall (R): {evaluation_results.mask.r.item():.3f}")
    print(f"  F1 Score: {2 * (evaluation_results.mask.p.item() * evaluation_results.mask.r.item()) / (evaluation_results.mask.p.item() + evaluation_results.mask.r.item()):.3f}")
    print(f"  mAP50 (Mean Average Precision at IoU=0.50): {evaluation_results.mask.map50.item():.3f}")
    print(f"  mAP50-95 (Mean Average Precision across IoU=0.50 to 0.95): {evaluation_results.mask.map.item():.3f}")
else:
    print("  Mask metrics not available (model may not be a segmentation model or no masks were predicted).")

# Accessing Speed Metrics
print("\nSpeed Metrics (per image):")
print(f"  Preprocess Time: {evaluation_results.speed['preprocess']:.1f}ms")
print(f"  Inference Time: {evaluation_results.speed['inference']:.1f}ms")
print(f"  Postprocess Time: {evaluation_results.speed['postprocess']:.1f}ms")
total_ms_per_image = evaluation_results.speed['preprocess'] + evaluation_results.speed['inference'] + evaluation_results.speed['postprocess']
if total_ms_per_image > 0:
    approx_fps = 1000 / total_ms_per_image
    print(f"  Approximate FPS: {approx_fps:.1f}")
else:
    print("  Approximate FPS: N/A (Total time per image is zero)")

print("\nEvaluation complete!")


--- Detailed Evaluation Results ---

Bounding Box Metrics:
  Precision (P): 0.932
  Recall (R): 0.951
  F1 Score: 0.941
  mAP50 (Mean Average Precision at IoU=0.50): 0.969
  mAP50-95 (Mean Average Precision across IoU=0.50 to 0.95): 0.661

Segmentation Mask Metrics:
  Mask metrics not available (model may not be a segmentation model or no masks were predicted).

Speed Metrics (per image):
  Preprocess Time: 0.3ms
  Inference Time: 25.5ms
  Postprocess Time: 2.9ms
  Approximate FPS: 34.9

Evaluation complete!


In [ ]:
model = YOLO('yolov8n-seg.pt')
print(f"DEBUG: Model's detected task: {model.task}")

100%|██████████| 6.74M/6.74M [00:00<00:00, 107MB/s]


DEBUG: Model's detected task: segment


In [ ]:

import os

# List files in /content
print(os.listdir('/content'))

['.config', 'sample_data']
